## Librerias 

In [ ]:
import numpy as np

# Para tratamiento y e/s de datos
import pandas as pd

# Gráficos de datos
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

#filtrado para suavizar los datos
from scipy.signal import savgol_filter

import time

In [ ]:
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import performance_metrics

## Forecasting: Demanda (Prophet) con Clima

In [ ]:
# Automatización: Importar los archivos generados por comb_e&w.
df = pd.read_csv(r'---INSERTAR ARCHIVO .CSV DE DATOS COMBINADOS---')

In [ ]:
#Convierto a tipo DateTimeIndex la columna "Datetime"
df['datetime'] = pd.to_datetime(df['datetime'])
df.sort_values(by=['datetime'], axis = 0, ascending = True, inplace = True)
df.reset_index(inplace = True, drop = True)

In [ ]:
print('\t\t\t\t\t\tINFORMACIÓN GENERAL \n')
print(f'{df.info()}\n')
print("Variable objetivo: y[kW]\n")

## Filtron de señal para suavizar puntos de cambios bruscos (vale la pena?)

In [ ]:
y_filtered = df[['y[kW]']].apply(savgol_filter,  window_length=5, polyorder=3)
y_filtered['temp'] = df['temp']
y_filtered['datetime'] = df['datetime']
y_filtered['datetime'] = pd.to_datetime(y_filtered['datetime'])
y_filtered.sort_values(by=['datetime'], axis = 0, ascending = True, inplace = True)
y_filtered.reset_index(inplace = True, drop = True)

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index,y=df['y[kW]'],
                         mode='lines',
                         name='No Filtrada'))
fig.add_trace(go.Scatter(x=y_filtered.index, y=y_filtered['y[kW]'],
                         mode='lines', 
                         name='Filtrada'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.show()

## Extraemos características de la variable Tiempo

<i>
Podemos dividir la columna de Datetime en sus diferentes componentes. <br>
Esto nos permite encontrar patrones para diferentes grupos.
</i>

In [ ]:
df_eda = df.copy()

In [ ]:
print(df_eda.head())
print('\n')
print(df_eda.info())

In [ ]:
df_eda['dow'] = df_eda['datetime'].dt.day_of_week
df_eda['doy'] = df_eda['datetime'].dt.day_of_year
df_eda['year'] = df_eda['datetime'].dt.year
df_eda['month'] = df_eda['datetime'].dt.month
df_eda['quarter'] = df_eda['datetime'].dt.quarter
df_eda['hour'] = df_eda['datetime'].dt.hour
df_eda['weekday'] = df_eda['datetime'].dt.day_name()
df_eda['woy'] = df_eda['datetime'].dt.isocalendar().week #week of year
df_eda['dom'] = df_eda['datetime'].dt.day # Day of Month
df_eda['date'] = df_eda['datetime'].dt.date 

# número de estación del año. El operador aritmético // solo devuelve a parte entera de la división.
df_eda['season'] = df_eda['month'].apply(lambda month_number: (month_number%12 + 3)//3) 

#Plotyle no permite acceso directo a los index del df. ?????
df_eda['date_and_time'] = df_eda.index 

In [ ]:
df_eda.head()

## EDA

### Graficando el consumo de energía a lo largo del tiempo

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_eda.datetime,y=df_eda['y[kW]'],
                         mode='lines',
                         name='Energia'))

# adjust layout
fig.update_traces(line=dict(width=1.5))
fig.show()

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_eda.datetime, y=df_eda['temp'],
                         mode='lines', 
                         name='Clima'))

# adjust layout
fig.update_traces(line=dict(width=1.5))
fig.show()

### Patrones de demanda

In [ ]:
#Podemos usar nuestras funciones de fecha y hora extraídas previamente 
#para ver si surgen patrones recurrentes de los datos agregados. 
#Tomemos, por ejemplo, la demanda de energía a lo largo del día para cada día de la semana:

In [ ]:
# Dataframe definido para reflejar el consumo por hora en la semana, usando la mediana de energia. 
patron_1 = df_eda.groupby(['hour', 'weekday'], as_index=False).agg({'y[kW]':'median'})

In [ ]:
# Dataframe definido para reflejar el consumo por hora en la semana, usando la mediana de energia. 
patron_2 = df_eda.groupby(['hour', 'weekday'], as_index=False).agg({'temp':'median'})

In [ ]:
# Dataframe definido para reflejar el consumo por hora en la semana, usando la mediana de energia. 
patron_3 = df_eda.groupby(['hour', 'season'], as_index=False).agg({'y[kW]':'median'})

In [ ]:
# Dataframe definido para reflejar el consumo por hora en la semana, usando la mediana de energia. 
patron_4 = df_eda.groupby(['hour', 'season'], as_index=False).agg({'temp':'median'})

In [ ]:
fig = px.line(patron_1, 
              x = 'hour',
              y = 'y[kW]', 
              color='weekday', 
              title='Mediana de demanda por hs por día de semana ')

fig.update_layout(xaxis_title='hs', yaxis_title='Demanda[kW]')

fig.show()

In [ ]:
fig_2 = px.line(patron_2, 
                x = 'hour',
                y = 'temp', 
                color='weekday', 
                title='Mediana de temperatura por hs por día de semana')

fig_2.update_layout(xaxis_title='hs', yaxis_title='T[ªC]')

fig_2.show()

In [ ]:
fig_3 = px.line(patron_3, 
                x = 'hour',
                y = 'y[kW]', 
                color='season', 
                title='Mediana de consumo de energia por hs por estación')

fig_3.update_layout(xaxis_title='hs', yaxis_title='Demanda[kW]')

fig_3.show()

In [ ]:
fig_4 = px.line(patron_4, 
                x = 'hour',
                y = 'temp', 
                color='season', 
                title='Mediana de temperatura por hs por estación')

fig_4.update_layout(xaxis_title='hs', yaxis_title='T[ªC]')

fig_4.show()

In [ ]:
df_eda[['month', 'temp']].groupby('month').agg({'temp':{'max', 'min', 'mean'}})

In [ ]:
df_eda[['y[kW]', 'temp']].corr()

In [ ]:
#Correlación del 68%. Bastante alta. 

## Partición de la serie de tiempo

### Opción 1º

In [ ]:
print(f'El primer punto de medicion fecha/hs es: {min(y_filtered.datetime)}')
print(f'El último punto de medicion fecha/hs es: {max(y_filtered.datetime)}')

In [ ]:
train = y_filtered.copy()
train.rename({'datetime':'ds','y[kW]':'y'},axis='columns',inplace = True)

In [ ]:
test = y_filtered.copy()
test.drop(['y[kW]'], axis=1, inplace = True)
test.rename({'datetime' :'ds'}, axis='columns', inplace = True)

In [ ]:
print(train.info())
print('\n')
print(test.info())

### Opcion alternativa para Re-Fitting: Divido los datos en un tiempo siempre constante para generar un pre-ajuste de datos a usar siempre

In [ ]:
train = y_filtered.copy()
train.rename({'datetime':'ds','y[kW]':'y'},axis='columns',inplace = True)

In [ ]:
recorte = y_filtered['datetime'][35040] #un año
first_year = y_filtered[y_filtered['datetime']<=recorte].copy()
first_year.rename({'datetime':'ds','y[kW]':'y'},axis='columns',inplace = True)

In [ ]:
test = train.copy()
test.drop(['y'], axis=1, inplace = True)

In [ ]:
print(train.info())
print('\n')
print(test.info())
print('\n')
print(first_year.info())

# Prophet

Es un modelo de pronóstico de series de tiempo, diseñado para manejar las características comunes
en las series de tiempo implementadas hoy en día. <br>
La idea del modelo Prophet es ser accesible y ajustable sin necesitar tener conocimientos de lo que pasa
detrás del telón respecto al funcionamiento matemático de la serie de tiempo. <br>
Tecnicamente hablando, es una serie de tiempo descompuesta en tres términos:
<i>y(t) = g(t)+s(t)+h(t)+et</i>
<ul>
<li>g(t): trend
    <blockquote> 
        Función de tendencia que modela cambios no-periodicos en los valores de la serie de tiempo.
    </blockquote>
    </li> 
<li>s(t): seasonality
    <blockquote>   
        Función que representa cambios periodicos. 
    </blockquote>
    </li> 
<li>h(t): holidays
    <blockquote>  
        Función que representa los efectos de los días de vacaciones/feriados/findes.
    </blockquote>
    </li>
<li>et: Término de error. 
    <blockquote>  
        Representa cualquier cambio idiosincracico (herencia). Se supone normalmente distribuido. 
    </blockquote>
    </li>
</ul>

Docs Oficiales (muy utiles): __[PROPHET_DOCS](https://facebook.github.io/prophet/docs/quick_start.html)__<BR>
Teoría: __[Forecasting at Scale(pdf)](https://www.kaggle.com/robinteuwens/forecasting-energy-demand/notebook)__ <br>
Practica: __[Forecasting con Prophet](https://nextjournal.com/eric-brown/forecasting-with-prophet)__ 

## Conditional Seasonalities

__Teoría (fundamentos): ['How does Prophet work?'](https://medium.com/analytics-vidhya/how-does-prophet-work-part-2-c47a6ceac511)__

<blockquote>
In some instances the seasonality may depend on other factors, such as a weekly seasonal pattern that is different during the summer than it is during the rest of the year, or a daily seasonal pattern that is different on weekends vs. on weekdays. These types of seasonalities can be modeled using conditional seasonalities.
</blockquote>

Del EDA podemos observar que la variación diaria en estaciones es mayor en Verano e Invierno (obviamente). 
Los patrones de datos tienen dependencia del clima. 

In [ ]:
# Condiciones
def is_spring(ds): 
    date = pd.to_datetime(ds)    
    return (date.month >= 3) & (date.month <=5)

def is_summer(ds): 
    date = pd.to_datetime(ds)
    return (date.month >= 6) & (date.month <=8)

def is_autumn(ds): 
    date = pd.to_datetime(ds)
    return (date.month >= 9) & (date.month <=11)

# La lógica fallaba, tuve que corregir. 
def is_winter(ds): 
    date = pd.to_datetime(ds)
    return (date.month == 12) | (date.month <=2)

# A esta función la hice de una forma distinta para que ande bien.
def is_weekend(ds):     
    return ds.dayofweek in (5, 6)

In [ ]:
# agregamos al set de entrenamiento
train['is_spring'] = train['ds'].apply(is_spring)
train['is_summer'] = train['ds'].apply(is_summer)
train['is_autumn'] = train['ds'].apply(is_autumn)
train['is_winter'] = train['ds'].apply(is_winter)
train['is_weekend'] = train['ds'].apply(is_weekend)
train['is_weekday'] = ~train['ds'].apply(is_weekend) 

In [ ]:
# agregamos al set de entrenamiento
test['is_spring'] = test['ds'].apply(is_spring)
test['is_summer'] = test['ds'].apply(is_summer)
test['is_autumn'] = test['ds'].apply(is_autumn)
test['is_winter'] = test['ds'].apply(is_winter)
test['is_weekend'] = test['ds'].apply(is_weekend)
test['is_weekday'] = ~test['ds'].apply(is_weekend) 

In [ ]:
# agregamos al set de entrenamiento
first_year['is_spring'] = first_year['ds'].apply(is_spring)
first_year['is_summer'] = first_year['ds'].apply(is_summer)
first_year['is_autumn'] = first_year['ds'].apply(is_autumn)
first_year['is_winter'] = first_year['ds'].apply(is_winter)
first_year['is_weekend'] = first_year['ds'].apply(is_weekend)
first_year['is_weekday'] = ~first_year['ds'].apply(is_weekend) 

## Definimos función MAPE: error de porcentaje absoluto medio

In [ ]:
def mape(y_true, y_pred):
    """Error de porcentaje absoluto medio"""
    
    # conversión a vectores numpy
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    # Porcentaje de error
    pe = (y_true - y_pred) / y_true
    
    # valor absolutos
    ape = np.abs(pe)
    
    # Cuantificación del rendimiento en un solo nº
    mape = np.mean(ape)
    
    return f'{mape*100:.2f}%'

In [ ]:
modelo = Prophet(
                 daily_seasonality = False,
                 weekly_seasonality = False,
                 yearly_seasonality = False,
             
)

modelo.add_seasonality(name='yearly', period=365.25, fourier_order = 15) #15

modelo.add_seasonality(name='weekly_spring', 
                        period=7,
                        fourier_order = 5, #5
                        condition_name='is_spring')
modelo.add_seasonality(name='weekly_summer', 
                        period=7,
                        fourier_order=5, 
                        condition_name='is_summer')
modelo.add_seasonality(name='weekly_autumn', 
                        period=7,
                        fourier_order=5, 
                        condition_name='is_autumn')
modelo.add_seasonality(name='weekly_winter', 
                        period=7,
                        fourier_order=5, 
                        condition_name='is_winter')

modelo.add_seasonality(name='daily_spring',  
                        period=1,
                        fourier_order=3, 
                        condition_name='is_spring')
modelo.add_seasonality(name='daily_summer',  
                        period=1,
                        fourier_order=3, 
                        condition_name='is_summer')
modelo.add_seasonality(name='daily_autumn',  
                        period=1,
                        fourier_order=3, 
                        condition_name='is_autumn')
modelo.add_seasonality(name='daily_winter',  
                        period=1,
                        fourier_order=3, 
                        condition_name='is_winter')
modelo.add_seasonality(name='daily_weekend',  
                        period=1,
                        fourier_order=3, 
                        condition_name='is_weekend')
modelo.add_seasonality(name='daily_weekday',  
                        period=1,
                        fourier_order=3, 
                        condition_name='is_weekday')

modelo.add_regressor(name = 'temp', standardize = False)
                        
# Feriados/días festivos
# modelo.add_country_holidays(country_name = 'AR')

In [ ]:
# fitting el modelo
modelo.fit(train)

In [ ]:
# Prediciendo valores
forecast = modelo.predict(test)

In [ ]:
# Gráfica para visualizar los puntos de cambios y la tendencia de la predicción
fig = modelo.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), modelo, forecast)

In [ ]:
#graficando los componentes de Prophet.predict
pd.plotting.register_matplotlib_converters()
_ = modelo.plot_components(forecast)

In [ ]:
# Concateno los datos pronosticados a futuro con todo los datos ya medidos. 
test_prophet = y_filtered.copy()
final_df = pd.concat((forecast['yhat'], test_prophet), axis = 1)
final_df = final_df[['datetime', 'temp', 'y[kW]', 'yhat']]
final_df

In [ ]:
# create figure of the complete dataframe
fig = go.Figure()
fig.add_trace(go.Scatter(x=final_df.ds, y=final_df.y,
                         mode='lines',
                         name='Test - Ground Truth'))
fig.add_trace(go.Scatter(x=final_df.ds, y=final_df.yhat,
                         mode='lines', 
                         name='Test - Prediction'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.update_layout(title='Prophet Forecast of Hourly Energy Demand',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# quantify accuracy
print(f'MAPE for Prophet\'s predictions: {mape(final_df.y, final_df.yhat)}')

## Ajustando Hiper-Parámetros del modelo

__Time-Serie & Hyperparameter: ['Tuning'](https://www.kaggle.com/manovirat/timeseries-using-prophet-hyperparameter-tuning/notebook#HyperParameter-Tuning-using-ParameterGrid)__

### ParameterGrid

In [ ]:
from sklearn.model_selection import ParameterGrid

#ajustar los valores de cada parámetro
param_grid = {
'holidays_prior_scale' : [0.5, 2, 5, 10],
'changepoint_prior_scale': [0.01, 0.1, 0.2, 0.35, 0.5], #[0.005, 0.01, 0.1, 0.2, 0.35, 0.5],
'seasonality_prior_scale': [0.005, 0.01, 0.1, 1.0, 3.0, 5.0, 10.0], #[0.005, 0.01, 0.1, 1.0, 3.0, 5.0, 10.0]
}

all_comb = ParameterGrid(param_grid)
rmses = []
sum = 0 
for p in all_comb:
    sum = sum + 1 

print(sum)

In [ ]:
# El siguiente bloque va a durar MUUUUUCHO TIEMPO haciendo ajustes de un modelo por cada combinación posible según se haya configurado la grilla del bloque anterior
model_parameters = pd.DataFrame(columns = ['MAPE','Parameters'])
for p in all_comb:
    # random.seed(0)
    comb_model = Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
                         holidays_prior_scale = p['holidays_prior_scale'],
                         seasonality_prior_scale = p['seasonality_prior_scale'],
                         n_changepoints = 75,
                         changepoint_range = 0.99,
                         seasonality_mode = 'multiplicative',
                         weekly_seasonality=False,
                         daily_seasonality = False,
                         yearly_seasonality = False,
                         growth = 'linear',
                         interval_width=0.95)
    comb_model.add_seasonality(name='yearly', period=365.25, fourier_order = 15)
    comb_model.add_seasonality(name='weekly_spring', 
                            period=7,
                            fourier_order = 5,
                            condition_name='is_spring')
    comb_model.add_seasonality(name='weekly_summer', 
                            period=7,
                            fourier_order=5,
                            condition_name='is_summer')
    comb_model.add_seasonality(name='weekly_autumn', 
                            period=7,
                            fourier_order=5,
                            condition_name='is_autumn')
    comb_model.add_seasonality(name='weekly_winter', 
                            period=7,
                            fourier_order=5,
                            condition_name='is_winter')
    comb_model.add_seasonality(name='daily_spring',  
                            period=1,
                            fourier_order=3, 
                            condition_name='is_spring')
    comb_model.add_seasonality(name='daily_summer',  
                            period=1,
                            fourier_order=3,
                            condition_name='is_summer')
    comb_model.add_seasonality(name='daily_autumn',  
                            period=1,
                            fourier_order=5,
                            condition_name='is_autumn')
    comb_model.add_seasonality(name='daily_winter',  
                            period=1,
                            fourier_order=3,
                            condition_name='is_winter')
    comb_model.add_seasonality(name='daily_weekend',  
                            period=1,
                            fourier_order=3,
                            condition_name='is_weekend')
    comb_model.add_seasonality(name='daily_weekday',  
                            period=1,
                            fourier_order=3,
                            condition_name='is_weekday')
    comb_model.add_regressor(name = 'temp', 
                                 prior_scale=5,
                                 standardize = 'auto')
    comb_model.add_country_holidays(country_name='AR')
    
    comb_model.fit(train)    

    forecast = comb_model.predict(test)
    forecast_cut = forecast[['ds','yhat']]   
   
    MAPE = mape(y_filtered['y[kW]'],forecast_cut['yhat'])  
    model_parameters = model_parameters.append({'MAPE':MAPE,'Parameters':p},ignore_index=True)  
    print(f'Mean Absolute Percentage Error(MAPE)------------------------------------{MAPE}')
    print(p)

In [ ]:
# Imprimo el resultado de todas las combinaciones para seleccionar la mejor. Ordenadas por error MAPE
# pd.set_option("display.max_rows", None, "display.max_columns", None,'display.max_colwidth', -1)
parameters = model_parameters.sort_values(by=['MAPE'])
parameters = parameters.reset_index(drop=True)
parameters.head()

### .fit() con los mejores valores

In [ ]:
## AJUSTA LOS VALORES ANTES DE CORRER 
'''
comb_model = Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
                     holidays_prior_scale = p['holidays_prior_scale'],
                     seasonality_prior_scale = p['seasonality_prior_scale'],
                     n_changepoints = 75,
                     changepoint_range = 0.99,
                     seasonality_mode = 'multiplicative',
                     weekly_seasonality=False,
                     daily_seasonality = False,
                     yearly_seasonality = False,
                     growth = 'linear',
                     interval_width=0.95)

MAPE	Parameters
0	13.16%	{'changepoint_prior_scale': 0.35, 'holidays_prior_scale': 5, 'seasonality_prior_scale': 0.01} --- The One!
1	13.16%	{'changepoint_prior_scale': 0.2, 'holidays_prior_scale': 5, 'seasonality_prior_scale': 0.01}
2	13.16%	{'changepoint_prior_scale': 0.5, 'holidays_prior_scale': 5, 'seasonality_prior_scale': 0.01}
3	13.20%	{'changepoint_prior_scale': 0.35, 'holidays_prior_scale': 0.5, 'seasonality_prior_scale': 0.01}
4	13.24%	{'changepoint_prior_scale': 0.5, 'holidays_prior_scale': 0.5, 'seasonality_prior_scale': 0.01}

'''

'''
modelo_tuneado = Prophet(growth= 'linear',   
                         n_changepoints = 75, 
                         changepoint_range=0.90, 
                         yearly_seasonality = False,
                         weekly_seasonality=False,
                         daily_seasonality = False,                                                 
                         seasonality_mode = 'multiplicative',
                         seasonality_prior_scale=0.01,     #rango recomendado: 0.01 to 10 
                         holidays_prior_scale=5,           #rango recomendado: * to 10
                         changepoint_prior_scale = 0.35,    #rango recomendado: 0.001 to 0.5                      
                         )
                         
Yearly F: 15 / Weakly F: 5 / Daily F: 3
9.91%  
'''
modelo_inicial = Prophet(growth= 'linear',   
                         n_changepoints = 75, 
                         changepoint_range=0.90, 
                         yearly_seasonality = True,
                         weekly_seasonality=False,
                         daily_seasonality = False,                                                 
                         seasonality_mode = 'multiplicative',
                         seasonality_prior_scale=0.01,     #rango recomendado: 0.01 to 10 
                         holidays_prior_scale=5,           #rango recomendado: * to 10
                         changepoint_prior_scale = 0.35,    #rango recomendado: 0.001 to 0.5                      
                         )

# modelo_tuneado.add_seasonality(name='yearly', period=365.25, fourier_order = 15)

modelo_inicial.add_seasonality(name='weekly_spring', 
                        period=7,
                        fourier_order = 5,
                        condition_name='is_spring')
modelo_inicial.add_seasonality(name='weekly_summer', 
                        period=7,
                        fourier_order=5,
                        condition_name='is_summer')
modelo_inicial.add_seasonality(name='weekly_autumn', 
                        period=7,
                        fourier_order=5,
                        condition_name='is_autumn')
modelo_inicial.add_seasonality(name='weekly_winter', 
                        period=7,
                        fourier_order=5,
                        condition_name='is_winter')

modelo_inicial.add_seasonality(name='daily_spring',  
                        period=1,
                        fourier_order=3, 
                        condition_name='is_spring')
modelo_inicial.add_seasonality(name='daily_summer',  
                        period=1,
                        fourier_order=3,
                        condition_name='is_summer')
modelo_inicial.add_seasonality(name='daily_autumn',  
                        period=1,
                        fourier_order=5,
                        condition_name='is_autumn')
modelo_inicial.add_seasonality(name='daily_winter',  
                        period=1,
                        fourier_order=3,
                        condition_name='is_winter')
modelo_inicial.add_seasonality(name='daily_weekend',  
                        period=1,
                        fourier_order=3,
                        condition_name='is_weekend')
modelo_inicial.add_seasonality(name='daily_weekday',  
                        period=1,
                        fourier_order=3,
                        condition_name='is_weekday')
                        
modelo_inicial.add_regressor(name = 'temp', 
                             prior_scale=5,
                             standardize = 'auto')

# Feriados/días festivos
modelo_inicial.add_country_holidays(country_name = 'AR')

In [ ]:
'''
NO EJECUTAR
'''
print(modelo_inicial.seasonalities.values())
print('\n')
for props in modelo_inicial.seasonalities.values():
    print(props['condition_name'])

In [ ]:
# fitting el modelo
start_1 = time.time()
modelo_inicial.fit(first_year)
end_1 = time.time() - start_1

In [ ]:
print(end_1)

In [ ]:
'''
NO EJECUTAR
'''
#graficando los componentes de Prophet.predict()
pd.plotting.register_matplotlib_converters()
_ = modelo_inicial.plot_components(forecast)

### Re-fitting

In [ ]:
De los datos bajados desde la API, debo:
    * dividirlos por año para un modelo "m1"
    * usar todo el set de datos para el modelo "m2" usando los parametros de ajuste del modelo "m1"

In [ ]:
res = {}
for pname in ['k', 'm', 'sigma_obs']:
    res[pname] = modelo_inicial.params[pname][0][0]
for pname in ['delta', 'beta']:
    res[pname] = modelo_inicial.params[pname][0]

In [ ]:
# '''
# NO EJECUTAR!
# '''
mod_final = Prophet(growth= 'linear',   
             n_changepoints = 75, 
             changepoint_range=0.90, 
             yearly_seasonality = True,
             weekly_seasonality=False,
             daily_seasonality = False,                                                 
             seasonality_mode = 'multiplicative',
             seasonality_prior_scale=0.005,     #rango recomendado: 0.01 to 10 
             holidays_prior_scale=5,           #rango recomendado: * to 10
             changepoint_prior_scale = 0.35,    #rango recomendado: 0.001 to 0.5                      
             )

# modelo_tuneado.add_seasonality(name='yearly', period=365.25, fourier_order = 15)

mod_final.add_seasonality(name='weekly_spring', 
                        period=7,
                        fourier_order = 5,
                        condition_name='is_spring')
mod_final.add_seasonality(name='weekly_summer', 
                        period=7,
                        fourier_order=5,
                        condition_name='is_summer')
mod_final.add_seasonality(name='weekly_autumn', 
                        period=7,
                        fourier_order=5,
                        condition_name='is_autumn')
mod_final.add_seasonality(name='weekly_winter', 
                        period=7,
                        fourier_order=5,
                        condition_name='is_winter')

mod_final.add_seasonality(name='daily_spring',  
                        period=1,
                        fourier_order=3, 
                        condition_name='is_spring')
mod_final.add_seasonality(name='daily_summer',  
                        period=1,
                        fourier_order=3,
                        condition_name='is_summer')
mod_final.add_seasonality(name='daily_autumn',  
                        period=1,
                        fourier_order=5,
                        condition_name='is_autumn')
mod_final.add_seasonality(name='daily_winter',  
                        period=1,
                        fourier_order=3,
                        condition_name='is_winter')
mod_final.add_seasonality(name='daily_weekend',  
                        period=1,
                        fourier_order=3,
                        condition_name='is_weekend')
mod_final.add_seasonality(name='daily_weekday',  
                        period=1,
                        fourier_order=3,
                        condition_name='is_weekday')
                        
mod_final.add_regressor(name = 'temp', 
                         prior_scale=5,
                         standardize = 'auto')

# Feriados/días festivos
mod_final.add_country_holidays(country_name = 'AR')

In [ ]:
start_2 = time.time()
mod_final.fit(train, init=res) # Adding the new data, warm-starting from modelo_tuneado
end_2 = time.time() - start_2

In [ ]:
print(end_2)

In [ ]:
# Prediciendo valores
forecast = mod_final.predict(test)

In [ ]:
# Gráfica para visualizar los puntos de cambios y la tendencia de la predicción
fig = mod_final.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), mod_final, forecast)

### Set de datos final

In [ ]:
test_prophet = y_filtered.copy()
final_df = pd.concat((forecast['yhat'], test_prophet), axis = 1)
final_df = final_df[['datetime', 'temp', 'y[kW]', 'yhat']]

In [ ]:
'''
NO EJECUTAR 
'''
# test_prophet = y_filtered[y_filtered['datetime']>start].copy()
# test_prophet.reset_index(inplace = True, drop = True)
# final_df = pd.concat((forecast_tunning['yhat'], test_prophet), axis = 1)
# final_df = final_df[['datetime', 'temp', 'y[kW]', 'yhat']]

## Graficamos los nuevos resultados obtenidos: Curva de test y de valores predecidos

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=final_df.datetime, y=final_df['y[kW]'],
                         mode='lines',
                         name='Test - Ground Truth'))
fig.add_trace(go.Scatter(x=final_df.datetime, y=final_df.yhat,
                         mode='lines', 
                         name='Test - Prediction'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.update_layout(title='Prophet Forecast of Hourly Energy Demand',
                  xaxis_title='Date & Time',
                  yaxis_title='Demand [MW]')
fig.show()

In [ ]:
# quantify accuracy
x = final_df['y[kW]']
y = final_df['yhat']
print(f'MAPE for Prophet\'s predictions: {mape(x,y)}')

## Guardo los datos obtenidos en un archivo .csv

In [ ]:
df_y = pd.DataFrame(final_df)
df_y.rename(columns={'ds':'datetime', 'y':'Demanda en [kW]', 'yhat': 'Demanda proyectada'}, inplace = True)
df_y.to_csv('---ARCHIVO FINAL .CSV CON DATOS PRONOSTICADOS---', index = False, encoding='utf-8')